<a href="https://colab.research.google.com/github/pantelisziazo/various_projects/blob/main/Weather_Forecast_MetaWeather_API_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Weather Forecast

##### **By Pantelis Ziazopoulos**
##### (Find me to larryziazo@gmail.com or my [LinkedIn Account](https://www.linkedin.com/in/pantelis-ziazopoulos-82a3b51ab/) )


##### Thank you [API - MetaWeather](https://www.metaweather.com/api/)

In the script that follows I will :


1.   Get n-day forecast of large cities around the globe
2.   Store the data in SQL databases (using colab)
3.   Make a DataFrame of the forecast for each location for every day
4.   Calculate the AVERAGE of the latest 3 forecast for each location for every day
5.   Get the top n locations based on each available numerical metric



### Import the necessary libraries and set problem variables


In [1]:
import json  
from pathlib import Path  
import requests  # run the command {!pip install requests} if 'requests' library is not installed
import numpy as np 
import pandas as pd
import datetime

# Input of the problem
NUM_CITIES = int(input("Give number of cities : "))
NUM_DAYS = int(input("Give number of the days ahead you want the forecast (including today): "))

# Get n-day ahead of today daystamps (max 10-day forecast, the website does not has information regarding 10 days ahead)
today = datetime.datetime.today()
seven_day_period =[]
for i in range(0, NUM_DAYS): 
  day = today + datetime.timedelta(days=i)
  seven_day_period.append(day.strftime('%Y/%m/%d'))


# Get the name of the city

# User input the name of the cities
city_names = []
for i in range(NUM_CITIES):
  city_names.append(input("Enter the name of a city (large cities only): "))

# Example given input 
# city_names = ['London', 'Paris', 'Athens']    


Give number of cities : 4
Give number of the days ahead you want the forecast (including today): 7
Enter the name of a city (large cities only): London
Enter the name of a city (large cities only): Berlin
Enter the name of a city (large cities only): Paris
Enter the name of a city (large cities only): Athens


In [2]:
# Get the ids codes for each city
# Credits : https://pythonandvba.com/blog/build-a-weather-app-in-excel-by-using-python/

URL_cities, response_cities, city_titles, city_ids = [], [], [], []
for i in city_names:
  URL_cities.append(f"https://www.metaweather.com/api/location/search/?query={i}")
  response_cities.append(requests.request("GET", URL_cities[-1])) # the city info (name, cordinates, id, location type)
  city_titles.append(json.loads(response_cities[-1].text)[0]["title"]) # the cities names
  city_ids.append(json.loads(response_cities[-1].text)[0]["woeid"]) # the woeid codes 


In [3]:
# A quick look at the format of the data till here
print("The codes (woeid) of the cities are : " , city_ids)
print("The days that the forecasts refer to are : ", seven_day_period)

The codes (woeid) of the cities are :  [44418, 638242, 615702, 946738]
The days that the forecasts refer to are :  ['2021/10/03', '2021/10/04', '2021/10/05', '2021/10/06', '2021/10/07', '2021/10/08', '2021/10/09']


### Get the forecasts of the chosen cities

In [4]:
# Get Weather information for each City
data = {}

for i,k in zip(city_ids,city_names):
  weather_location  = {}
  
  for j in seven_day_period:
    # weather for a specific location for specific date
    weather_location_date = "https://www.metaweather.com/api/location/" + str(i) + "/" + j  
    get_weather_location_date = requests.request("GET", weather_location_date).json()
    weather_location["{0}_weather_location".format(j)] = pd.DataFrame(get_weather_location_date)
  
  # axis=0 -> concatanation of x-axis   # ignore index so that the dates are ignored
  data["{0}_weather".format(k)] = pd.concat(weather_location,axis=0, ignore_index=True) 
  

In [5]:
# remove the unecessary strings from datetime values 
# the datetime value at 'created' column holds information probably from the timezone of the city 
# -> that is not needed for the purposes of this task that is why this pice of data is omitted
# -> also the all datetimes are converted to the same format 

for i in data.keys():
  for j in range(len(data[i]['created'])):
    data[i]['created'][j] = datetime.datetime. strptime(data[i].loc[j,'created'].split(".")[0], '%Y-%m-%dT%H:%M:%S')
    data[i]['applicable_date'][j] = datetime.datetime. strptime(data[i].loc[j,'applicable_date'], '%Y-%m-%d')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
# import the library below in order to 'pretty-print' more than 1 panda DataFrames
from IPython.core import display as ICD

data_keys = list(data.keys())
for i in data_keys:
  print(i)
  ICD.display(data[i].head())
  print("\n\n\n")    

London_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,6387646096998400,Light Rain,lr,SW,2021-10-03 15:59:01,2021-10-03 00:00:00,10.600,15.850,15.640,8.754025,225.666535,998.0,68,12.047455,75
1,5495230276042752,Light Rain,lr,SW,2021-10-03 12:59:03,2021-10-03 00:00:00,10.645,16.015,14.400,8.614009,227.333465,997.0,75,11.977551,75
2,5479289236488192,Light Rain,lr,SW,2021-10-03 09:59:02,2021-10-03 00:00:00,10.565,16.015,14.515,9.690802,227.666907,997.5,71,13.347985,75
3,5611871194316800,Light Rain,lr,SW,2021-10-03 06:59:01,2021-10-03 00:00:00,10.335,16.355,15.880,9.923609,225.666366,998.5,62,14.011610,75
4,6524113154736128,Light Rain,lr,SW,2021-10-03 03:59:01,2021-10-03 00:00:00,10.375,16.375,15.845,9.974399,225.333228,998.5,62,14.955162,75






Berlin_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,4897642191519744,Heavy Cloud,hc,SSE,2021-10-03 16:02:52,2021-10-03 00:00:00,13.340,21.71,19.270,8.418021,159.166641,1007.5,61,15.232915,71
1,6005044504690688,Heavy Cloud,hc,SSE,2021-10-03 13:02:53,2021-10-03 00:00:00,13.340,20.92,19.580,8.418021,159.166641,1006.5,67,15.232915,71
2,6129260864471040,Heavy Cloud,hc,SSE,2021-10-03 10:02:53,2021-10-03 00:00:00,13.305,20.68,19.540,8.276174,158.833346,1006.5,67,14.245245,71
3,4547760666705920,Heavy Cloud,hc,SSE,2021-10-03 07:02:52,2021-10-03 00:00:00,13.335,20.54,19.315,8.293987,158.833346,1008.0,60,14.480745,71
4,6393319723630592,Heavy Cloud,hc,SSE,2021-10-03 04:02:51,2021-10-03 00:00:00,13.575,20.47,19.330,8.230654,158.500013,1008.0,61,14.540707,71






Paris_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,5927472831922176,Heavy Rain,hr,SW,2021-10-03 13:19:05,2021-10-03 00:00:00,12.985,17.900,16.445,8.945057,226.468361,1000.5,88,9.477153,77
1,4591669283389440,Heavy Rain,hr,SW,2021-10-03 10:19:05,2021-10-03 00:00:00,12.810,17.960,16.395,8.536626,227.617354,1000.5,88,8.528320,77
2,6490003757596672,Heavy Rain,hr,SW,2021-10-03 07:19:04,2021-10-03 00:00:00,12.485,17.575,15.240,8.693211,227.617354,1003.5,84,8.163264,77
3,5160165893472256,Heavy Rain,hr,SW,2021-10-03 04:19:03,2021-10-03 00:00:00,12.465,17.460,15.255,8.842421,227.234097,1003.5,84,8.179420,77
4,6090915396452352,Heavy Rain,hr,SSW,2021-10-03 01:19:05,2021-10-03 00:00:00,12.230,18.580,15.620,10.578941,197.616640,1006.5,80,12.180429,77






Athens_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,5333048796119040,Light Cloud,lc,NNE,2021-10-03 13:59:35,2021-10-03 00:00:00,14.780,23.125,21.960,8.230444,13.827795,1021.0,51,14.545678,70
1,5770682634862592,Light Cloud,lc,NNE,2021-10-03 10:59:35,2021-10-03 00:00:00,14.750,23.180,22.095,8.058156,13.827795,1021.0,51,14.220080,70
2,5870909052157952,Clear,c,NNE,2021-10-03 07:59:34,2021-10-03 00:00:00,15.430,23.470,22.145,8.117393,12.487362,1021.5,48,14.392510,68
3,4781259608817664,Clear,c,NNE,2021-10-03 04:59:34,2021-10-03 00:00:00,15.430,23.470,22.150,8.087567,12.487362,1021.5,48,14.412083,68
4,6465711422570496,Heavy Cloud,hc,N,2021-10-03 01:59:34,2021-10-03 00:00:00,15.255,24.145,21.865,8.989110,6.000000,1020.5,49,13.506746,71


### Save the weather data to SQL databases (using SQLite3 which is compatible with colab)

In [7]:
# Credits : https://www.youtube.com/watch?v=DihBLvfXVSE
# Credits : https://colab.research.google.com/github/lucasmoratof/sql-projects/blob/master/Introduction_to_SQL_sub_queries.ipynb#scrollTo=_xkGEn95uh6W

import sqlite3

databases = {}
for i in range(NUM_CITIES):
  # open the sql and connect to a database
  databases["{0}_data".format(city_names[i])] = sqlite3.connect('{0}.db'.format(city_names[i]))
  
  # index = False -> drop the datetime from the index   
  # if_exists='replace -> to renew the database every time it is called
  data[data_keys[i]].to_sql('{0}_table'.format(city_names[i]), 
                            databases["{0}_data".format(city_names[i])], if_exists='replace', index=False)
  
  # commit the changes
  databases["{0}_data".format(city_names[i])].commit()


In [8]:
# example that sql command and pandas commands give same output
example_average = databases["{0}_data".format(city_names[0])].execute(
    '''SELECT AVG(max_temp) AS avg_temp FROM {0}_table;'''.format(city_names[0]))

for j in example_average:
  print(j[0] == data[data_keys[0]]['max_temp'].mean())


True


In [9]:
# print a nice sql table ! 
pd.read_sql_query("SELECT * FROM {0}_table".format(city_names[0]), databases["{0}_data".format(city_names[0])]).head()
# you can run { print(data[data_keys[0]].head()) } in order to check that the 2 dataframes are the same 


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,6387646096998400,Light Rain,lr,SW,2021-10-03 15:59:01,2021-10-03 00:00:00,10.600,15.850,15.640,8.754025,225.666535,998.0,68,12.047455,75
1,5495230276042752,Light Rain,lr,SW,2021-10-03 12:59:03,2021-10-03 00:00:00,10.645,16.015,14.400,8.614009,227.333465,997.0,75,11.977551,75
2,5479289236488192,Light Rain,lr,SW,2021-10-03 09:59:02,2021-10-03 00:00:00,10.565,16.015,14.515,9.690802,227.666907,997.5,71,13.347985,75
3,5611871194316800,Light Rain,lr,SW,2021-10-03 06:59:01,2021-10-03 00:00:00,10.335,16.355,15.880,9.923609,225.666366,998.5,62,14.011610,75
4,6524113154736128,Light Rain,lr,SW,2021-10-03 03:59:01,2021-10-03 00:00:00,10.375,16.375,15.845,9.974399,225.333228,998.5,62,14.955162,75


### Get the latest forecast for each location for every day 

In [10]:
# in order for the days to be in a sorted order
set_of_days = list(sorted(set(data[data_keys[0]]['applicable_date'])))

first_answer = {}

# for each location
for i in data_keys:
  latest_forecast_location_all_days = []
  
  # for each day
  for j in range(NUM_DAYS):
    forecast_location_day = data[i].loc[data[i]['applicable_date'] == set_of_days[j]]
    
    # get the latest forecast
    latest_forecast_location_all_days.append(
        forecast_location_day.loc[forecast_location_day['created'] == max(forecast_location_day['created'])])
  
  # concatenate the results of different days but for the same location 
  first_answer[i] = pd.concat(latest_forecast_location_all_days, axis=0, ignore_index=True)


In [11]:
# pretty-print the results
for i in first_answer.keys():
  print(i)
  ICD.display(first_answer[i])
  print("\n\n\n")    

London_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,6387646096998400,Light Rain,lr,SW,2021-10-03 15:59:01,2021-10-03 00:00:00,10.600,15.850,15.640,8.754025,225.666535,998.0,68,12.047455,75
1,5050488291590144,Showers,s,SSW,2021-10-03 15:59:01,2021-10-04 00:00:00,11.225,16.770,16.210,8.833452,213.320321,1009.5,66,11.784615,73
2,4922095050948608,Light Rain,lr,SW,2021-10-03 15:59:01,2021-10-05 00:00:00,10.640,14.550,14.480,11.675896,224.171894,1001.0,74,11.138079,75
3,5756485058428928,Heavy Cloud,hc,WNW,2021-10-03 15:59:01,2021-10-06 00:00:00,9.220,15.880,15.705,9.492959,294.348774,1019.5,68,15.875723,71
4,5126787848732672,Heavy Cloud,hc,S,2021-10-03 15:59:01,2021-10-07 00:00:00,9.930,16.220,16.375,3.655460,184.999696,1026.0,80,14.910423,71
5,5177555008094208,Light Cloud,lc,ESE,2021-10-03 15:59:04,2021-10-08 00:00:00,9.205,17.830,16.360,2.425065,118.000000,1031.0,72,9.999727,70
6,6112441772539904,Light Cloud,lc,ENE,2021-10-03 15:59:07,2021-10-09 00:00:00,8.805,17.785,16.220,3.566865,60.500000,1033.0,77,9.999727,70






Berlin_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,4897642191519744,Heavy Cloud,hc,SSE,2021-10-03 16:02:52,2021-10-03 00:00:00,13.340,21.710,19.270,8.418021,159.166641,1007.5,61,15.232915,71
1,5751907990634496,Showers,s,W,2021-10-03 16:02:55,2021-10-04 00:00:00,11.885,17.465,16.355,4.692793,280.148066,1018.0,70,11.092097,73
2,5326951150518272,Light Rain,lr,SE,2021-10-03 16:02:58,2021-10-05 00:00:00,11.425,18.500,16.280,6.645668,144.601646,1012.0,77,12.536785,75
3,5410696671002624,Light Rain,lr,SW,2021-10-03 16:03:00,2021-10-06 00:00:00,10.735,15.615,14.210,5.261773,216.745572,1016.0,80,14.898306,75
4,4904647710998528,Heavy Cloud,hc,NNE,2021-10-03 16:03:03,2021-10-07 00:00:00,9.090,17.085,15.385,4.570609,22.171085,1026.0,73,15.423054,71
5,6744650489528320,Light Cloud,lc,NE,2021-10-03 16:03:06,2021-10-08 00:00:00,8.545,16.940,14.020,3.916871,55.000000,1032.0,71,9.999727,70
6,5421561092767744,Light Cloud,lc,NE,2021-10-03 16:03:10,2021-10-09 00:00:00,7.440,15.910,11.940,3.268079,39.500000,1035.0,63,9.999727,70






Paris_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,5927472831922176,Heavy Rain,hr,SW,2021-10-03 13:19:05,2021-10-03 00:00:00,12.985,17.900,16.445,8.945057,226.468361,1000.5,88,9.477153,77
1,5672211894501376,Light Cloud,lc,SSW,2021-10-03 13:19:08,2021-10-04 00:00:00,10.540,19.015,17.750,7.508741,204.165963,1016.0,68,12.528397,70
2,6046372265459712,Heavy Rain,hr,SSW,2021-10-03 13:19:11,2021-10-05 00:00:00,11.350,16.345,15.350,10.323098,210.226595,1006.5,82,12.180429,77
3,6427256969756672,Light Cloud,lc,W,2021-10-03 13:19:15,2021-10-06 00:00:00,9.550,16.710,16.710,8.796335,280.822558,1023.0,65,13.903491,70
4,5483422312038400,Clear,c,NNE,2021-10-03 13:19:18,2021-10-07 00:00:00,8.130,17.735,16.995,3.664458,32.500000,1028.0,68,13.075514,68
5,6572850967216128,Heavy Cloud,hc,NE,2021-10-03 13:19:20,2021-10-08 00:00:00,7.720,18.190,17.110,4.261192,49.000000,1028.0,69,9.999727,71
6,4857822945738752,Clear,c,NE,2021-10-03 13:19:23,2021-10-09 00:00:00,8.770,18.565,17.450,4.596192,50.500000,1026.0,68,9.999727,68






Athens_weather


,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability
0,5333048796119040,Light Cloud,lc,NNE,2021-10-03 13:59:35,2021-10-03 00:00:00,14.780,23.125,21.960,8.230444,13.827795,1021.0,51,14.545678,70
1,4891938206515200,Light Cloud,lc,NNE,2021-10-03 13:59:38,2021-10-04 00:00:00,14.765,24.765,23.535,6.779237,11.500000,1021.5,49,14.342490,70
2,5797542127206400,Clear,c,NNE,2021-10-03 13:59:41,2021-10-05 00:00:00,14.500,24.905,23.250,6.668181,27.000000,1022.0,53,14.283149,68
3,5289515569643520,Clear,c,NNE,2021-10-03 13:59:44,2021-10-06 00:00:00,14.575,24.160,22.040,6.561079,19.500000,1021.0,55,14.204856,68
4,5758753900068864,Heavy Cloud,hc,SE,2021-10-03 13:59:47,2021-10-07 00:00:00,15.280,24.090,22.605,4.675909,137.844358,1018.0,58,13.952890,71
5,4848816734863360,Heavy Cloud,hc,SSW,2021-10-03 13:59:50,2021-10-08 00:00:00,17.455,25.025,23.530,3.751621,194.000000,1019.0,65,9.999727,71
6,6320059761819648,Clear,c,NNE,2021-10-03 13:59:53,2021-10-09 00:00:00,16.455,23.300,22.800,5.245037,22.500000,1018.0,70,9.999727,68


### Calculate the AVERAGE of the latest 3 forecast for each location for every day for a metric of your choice

In [12]:
# these are the numerical metrics that average, max, or min matters (e.g. the average of 'id' column does not hold particular meaning)
options_metrics = ['min_temp','max_temp','the_temp','wind_speed','wind_direction','air_pressure','humidity','visibility']

# choose one metric from the given ones
def user_choose(options):
  print("Please choose:")
  for idx, element in enumerate(options):
    print("{}) {}".format(idx+1,element))
  print("\n")
  i = input("Enter number: ")
  try:
    if 0 < int(i) <= len(options):
      print("Metric chosen: ", options[int(i)-1])
      return options[int(i)-1]
  except:
    pass
  return options[int(i)-1]


In [13]:
# chose the metric
average_metric = user_choose(options_metrics)

Please choose:
1) min_temp
2) max_temp
3) the_temp
4) wind_speed
5) wind_direction
6) air_pressure
7) humidity
8) visibility


Enter number: 3
Metric chosen:  the_temp


In [14]:
second_answer = {}

# for each location
for i in data_keys:
  latest_3_forecast_location_all_days = []
  
  # for each day
  for j in range(NUM_DAYS):
    forecast_location_day = data[i].loc[data[i]['applicable_date'] == set_of_days[j]]
    
    # get the average of the 3 latest measurements of the chosen metric 
    # -> 3 decimal digit prediction was inserted just for "uniform and nice" printed output
    latest_3_forecast_location_all_days.append(format(forecast_location_day[:3][average_metric].mean(), '.3f'))
  
  # save the prediction according to the location
  second_answer[i] = latest_3_forecast_location_all_days

In [15]:
import pprint
pprint.pprint(second_answer)

{'Athens_weather': ['22.067',
                    '23.633',
                    '23.238',
                    '22.073',
                    '22.442',
                    '23.613',
                    '21.500'],
 'Berlin_weather': ['19.463',
                    '16.025',
                    '16.677',
                    '14.720',
                    '15.170',
                    '14.280',
                    '12.933'],
 'London_weather': ['14.852',
                    '16.133',
                    '14.518',
                    '15.785',
                    '17.153',
                    '16.787',
                    '16.460'],
 'Paris_weather': ['16.027',
                   '17.825',
                   '15.733',
                   '16.677',
                   '17.112',
                   '17.307',
                   '17.887']}


### Get the top n locations based on an available metric

In [16]:
# chose the metric
top_n_metric = user_choose(options_metrics)

Please choose:
1) min_temp
2) max_temp
3) the_temp
4) wind_speed
5) wind_direction
6) air_pressure
7) humidity
8) visibility


Enter number: 4
Metric chosen:  wind_speed


In [17]:
# chose the number of the n-top cities 
num_locations = input("Give the n : ")

Give the n : 3


In [18]:
max_values = {}

# get the maximum values of the chosen metric for each city
for i in data_keys:
  max_values[i] = max(data[i][top_n_metric])

third_answer = []

# sort the max_values in descending order 
sorted_max_values = sorted(max_values, key=max_values.get, reverse=True)

# keep only the n-top location
for r in sorted_max_values[:int(num_locations)]:
  third_answer.append([r, max_values[r]])

# print the output
for i in third_answer:
  print(i)

['Paris_weather', 12.201971494795728]
['London_weather', 11.87142093290384]
['Berlin_weather', 9.776486387648134]


### Summary

In order to complete the task above I work **9 hours**.

I implemented this script completely in **Colab** so that is affordable and **easy to run to anyone**. The user has only to press 'Run All' and give the necessary inputs.

The process I followed was (roughly speaking):
> 1.   I made a quick search at the Internet for starter pieces of related code since I did not know well how to use API and scrape data from internet sites. Also, maybe a solution is already out there ! :-P 
> 11.  After I downloaded the data, took a look it, arranged the possible unnecessary formats and searched for commands that made the outputs of the cells looks "pretty", user-friendly.
> 2.   **1st problem** : The easy part would be to make a 5-day forecast since there was a command for this purpose. But I wanted to make it more general (ideally for 7-day period). That is why I had to scrape the necessary data for each single day.
>3.    **2nd problem** : This script works only for big cities, cities that the API-MetaWeather site has information for them. This script scrapes the data according to the name of the city, not the cordinates of the city. That means that if you insert a name of city like 'Karditsa' , it will not work. 
>4.   Then I checked the format of the DataFrame is the appropriate one (The data in the DataFrames are ok, the values are stored in the same way to the SQL databases, the SQL databases can be read from the script and run future queries).
>5.   **3rd Problem** : Do not run this script many times in the same day for many sites or many days. Probably the MetaWeather site has a limitation to the data you can scrape. Solution: I experimented with just 2 cities for a 3-day forecast till the whole code was ready. 
>6.   I spent some time in the generalization of the script. I wanted the user to be able to chose the number of cities, the days of the forecast, the metric that the comparisons would be based upon. But at the same time I wanted the names of the variables (especially for the SQL databases) be characteristic to the city each one referred to. 

**Possible Future work** :  

*   Raise error to the user and make him give another input in the case of input values not being of the necesary/expected type and range (e.g. integer is expected and the user inserts a string). 
*   Make the code in such a way that you insert the name of a city and it calculates the weather of the nearest city that the MetaWeather site has information for it. However, for this purpose additional coordinate-related data from other sites will be necessary.

---

I hope that you find this script quite readable and understandable.

